In [ ]:
# !pip install openai

In [1]:
from openai import OpenAI

## 🔐 Inicialización segura del cliente OpenAI con variable de entorno

In [4]:
# AI_Queries/code_explanation/oaib/v01-script01.md
#   AI_Queries/code_explanation/oaib/v01-script01_Tipos_de_modelos_de_OpenAI.md
#   AI_Queries/code_explanation/oaib/v01-script01_diferencias“Modelos_de_Chat”y“Modelos_de_Completions”.md

import os
from openai import OpenAI

# Recuperar la clave API de la variable de entorno
api_key_environ = os.environ.get("OPENAI_API_KEY")
 
# Verificar que la clave API esté disponible
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada o está vacía.")
 
# Inicializar el cliente de OpenAI con la clave API
client_openai = OpenAI(api_key=api_key_environ)
 
# Usar el cliente para tus tareas
print("¡Cliente de OpenAI inicializado correctamente!")

¡Cliente de OpenAI inicializado correctamente!


## Ejemplo básico de Chat Completion

Primera llamada al endpoint `chat.completions.create`, sin parámetros extra.

In [ ]:
# AI_Queries/knowledge/prompt_AI_GPT-Chat_completions_API_parameter.md
# AI_Queries/knowledge/prompt_AI_GPT-Chat_Completions_API.md
# AI_Queries/knowledge/prompt_AI_GTP-CodeExplanation01-chat_completions_API.md
# AI_Queries/code_explanation/oaib/v01-script01_diferencias“Modelos_de_Chat”y“Modelos_de_Completions”.md
# AI_Queries/code_explanation/oaib/v01-script01_estructura_interna_endpoint_de_Chat_Completions.md
# AI_Queries/code_explanation/oaib/v01-script01_rangos-temperature,top_p,frequency_penalty,presence_penalty.md
# AI_Queries/code_explanation/oaib/v01-script01_temperature,top_p,frequency_penalty,presence_penalty.md
# AI_Queries/code_explanation/oaib/v01-script01_Tipos_de_modelos_de_OpenAI.md

response = client_openai.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role": "system", "content" :"Eres un asistente que da informacion a dudas"},
        {"role": "user", "content" :"¿Quién descubrió América?"}
    ]
)

print(response.choices[0].message.content)

América fue descubierta por Cristóbal Colón, un navegante genovés al servicio de los Reyes Católicos de España, el 12 de octubre de 1492.


## Chat Completion con límite de tokens (max_tokens)

Se añade `max_tokens` para restringir la longitud de la respuesta.

In [7]:
response = client_openai.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role": "system", "content" :"Eres un asistente que da informacion a dudas"},
        {"role": "user", "content" :"¿Quién descubrió América?"}
    ],
    max_tokens = 50,
)

print(response.choices[0].message.content)

América fue descubierta por Cristóbal Colón en 1492, en su primer viaje financiado por los reyes de España. Sin embargo, es importante recordar que el continente ya estaba habitado por diversas cult


**Respuesta truncada**  
El texto finaliza abruptamente en “cult…”, lo que indica que se alcanzó el límite de tokens permitido (o se cortó por defecto).

## Chat Completion con muestreo y respuestas múltiples

Se incorporan `temperature`, `top_p` y `n=2` para controlar la aleatoriedad y obtener dos salidas.

In [14]:
response = client_openai.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role": "system", "content" :"Eres un asistente que da informacion a dudas"},
        {"role": "user", "content" :"¿Quién descubrió América?"}
    ],
    max_tokens = 50,
    temperature = 1,
    top_p = 1,
    n = 2
)
# n=2 produce la generación de dos respuestas
print(f"choices[0] es \t:{response.choices[0].message.content}\n")
print(f"choices[1] es \t:{response.choices[1].message.content}\n")
print(response)


choices[0] es 	:América fue descubierta por Cristóbal Colón en 1492 durante su primer viaje financiado por los Reyes Católicos de España.

choices[1] es 	:América fue descubierta por Cristóbal Colón en 1492, en su expedición financiada por los Reyes Católicos de España.

ChatCompletion(id='chatcmpl-BfsyWW0FpPdGEzv6P9sT7aBWewidG', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='América fue descubierta por Cristóbal Colón en 1492 durante su primer viaje financiado por los Reyes Católicos de España.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[])), Choice(finish_reason='stop', index=1, logprobs=None, message=ChatCompletionMessage(content='América fue descubierta por Cristóbal Colón en 1492, en su expedición financiada por los Reyes Católicos de España.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1749322472, model='gpt-3.5

Estas son algunas anotaciones específicas tras revisar la salida de:

* **Mínima diversidad entre elecciones**
  A pesar de haber usado `n=2` y `temperature=1`, ambas respuestas son prácticamente idénticas.

  * Implicación: esta pregunta factual es “de baja entropía”, por lo que el modelo tiende a repetir la misma formulación.
  * Acción sugerida: si necesitas variaciones más notables, podrías usar prompts más abiertos, subir `temperature` aún más (por ejemplo 1.2 en modelos que lo permitan) o ajustar `top_p` (por ejemplo a 0.8).

* **Terminación normal (`finish_reason='stop'`)**
  El hecho de que ambas elecciones terminen con `finish_reason='stop'` indica que la generación se completó al alcanzar una secuencia de parada o al concluir naturalmente, no por límite de tokens.

* **Tokens usados y coste**

  ```
  prompt_tokens=32  
  completion_tokens=67  
  total_tokens=99  
  ```

  Registra estos valores para estimar el coste y ajustar `max_tokens` en futuras pruebas (por ejemplo si quisieras respuestas más largas o más cortas).

* **Modelo en uso**
  Se informa `model='gpt-3.5-turbo-0125'`, lo cual confirma que, aunque especificaste `"gpt-3.5-turbo"`, el sistema ha concretado la versión `-0125`. Sirve para dejar constancia de la versión exacta de modelo probada.


In [16]:
# mejor forma de presentar el resultado anterior
for i, choice in enumerate(response.choices):
    print(f"Respuesta {i+1}: {choice.message.content}")


Respuesta 1: América fue descubierta por Cristóbal Colón en 1492 durante su primer viaje financiado por los Reyes Católicos de España.
Respuesta 2: América fue descubierta por Cristóbal Colón en 1492, en su expedición financiada por los Reyes Católicos de España.


## Chat Completion con temperatura baja y respuestas múltiples

Igual que el anterior, pero con `temperature=0.3` para respuestas más deterministas (menos “creativas”).

In [ ]:
response = client_openai.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role": "system", "content" :"Eres un asistente que da informacion a dudas"},
        {"role": "user", "content" :"¿Qué es IA?"}
    ],
    max_tokens = 100,
    temperature = 0.3,
    top_p = 1,
    n = 1
)

print(response.choices[0].message.content)

IA son las siglas de Inteligencia Artificial, que se refiere a la simulación de procesos de inteligencia humana por parte de máquinas, especialmente sistemas informáticos. La IA se utiliza en una amplia variedad de aplicaciones, como reconocimiento de voz, análisis de datos, conducción autónoma, entre otros.


Algunas anotaciones tras revisar la salida:

* **Alta coherencia y determinismo**
  Con `temperature=0.3` el modelo genera respuestas muy parecidas y estables. En preguntas definidas como “¿Qué es IA?”, resulta acertado porque da un enunciado claro y conciso.

* **Respuesta completa y sin truncar**
  El contenido está bien formado y no se ha cortado en medio de una frase; por tanto, `max_tokens=100` fue suficiente en este caso.

>* **Validar `finish_reason`**
  Comprueba que `response.choices[0].finish_reason == "stop"` para confirmar que terminó de forma natural y no por límite de tokens.

>* **Registro de uso de tokens**
  Extrae `response.usage` para anotar cuántos tokens consumió el prompt y la respuesta. Esto te ayudará a optimizar costes en futuras pruebas.


In [18]:
response.choices[0].finish_reason == "stop"

True

In [19]:
response.usage

CompletionUsage(completion_tokens=148, prompt_tokens=29, total_tokens=177, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))